# Running a program remotely

In this tutorial, we will write a basic program using Quantum Serverless. We will show how to run the program remotely and retrieve the results from the serverless client.

### Writing the Program

First, we need to write the program code and save it to a file called [program_1.py](./source_files/program_1.py). This program creates a two-qubit quantum circuit that prepares a Bell state, measures the result, and saves the measured probability distribution.

The code for the program is shown below:

```python
# source_files/program_1.py

from qiskit import QuantumCircuit
from qiskit.primitives import Sampler

from quantum_serverless import save_result

# Create a circuit
circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)
circuit.measure_all()
circuit.draw()

# Instantiate a Sampler to generate a quasi-distribution of the circuit's outputs
sampler = Sampler()

# Run the circuit and retrieve the quasi-distribution
quasi_dists = sampler.run(circuit).result().quasi_dists

# Save the result to the serverless client
save_result(quasi_dists)
```

### Running the Program

To run the program, we need to import the necessary classes and configure them. One of these classes is QuantumServerless, which is a client class for interacting with compute resources.

QuantumServerless takes a Provider object as a constructor argument. The Provider object stores configuration information about our compute resources, such as where they are located and how to connect to them. In this example, we will use a provider that is connected to a local Docker Compose setup. In this case, it allows us to run the program locally on our machine. If you want to run the program elsewhere, you will need to provide the corresponding host and authentication details.

In [1]:
from quantum_serverless import QuantumServerless, Provider
import os

/bin/bash: line 1: unzip: command not found


In [2]:
provider = Provider(
    username="user",
    password="password123",
    host=os.environ.get("GATEWAY_HOST", "http://localhost:8000"),
)

serverless = QuantumServerless(provider)
serverless

<QuantumServerless | providers [gateway-provider]>

In [3]:
dir(serverless)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_allocated_context',
 '_providers',
 '_selected_provider',
 'add_provider',
 'context',
 'delete',
 'download',
 'files',
 'get_job_by_id',
 'get_jobs',
 'job_client',
 'provider',
 'providers',
 'run',
 'set_provider',
 'upload',
 'widget']

After importing the necessary classes and configuring them, we can run the program by calling the `run()` method of the [QuantumServerless](https://qiskit-extensions.github.io/quantum-serverless/stubs/quantum_serverless.QuantumServerless.html#quantum_serverless.QuantumServerless) object:

[Program](https://qiskit-extensions.github.io/quantum-serverless/stubs/quantum_serverless.core.Program.html#quantum_serverless.core.Program) accepts couple of required parameters:
- title - name of the program
- entrypoint - name of python file you want to execute
- working_dir - directory where  your script is located (directory size must be less than 50MB). This is optional parameter and will be current folder by default.



In [ ]:
# -*- coding: utf-8 -*-
"""vqe.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1sDlUpjnsF0zYQRK5V06mhTDnvb-8nIhS
"""


import pickle
from qiskit.circuit.library import RealAmplitudes
from qiskit.algorithms.optimizers import COBYLA
# from qiskit_algorithms import COBYLA
from qiskit.providers.fake_provider import FakeKolkata
# from qiskit_aer import AerSimulator
# from qiskit_aer.noise import NoiseModel
from qiskit.result import ProbDistribution
from qiskit.opflow import I, X, Y, Z
from qiskit_ibm_runtime import Options

from quantum_serverless import Program

# relevant imports for the noise model
from qiskit import IBMQ

from circuit_knitting_toolbox.circuit_cutting.wire_cutting import cut_circuit_wires, reconstruct_full_distribution, evaluate_subcircuits

# Runtime imports
from qiskit_ibm_runtime import QiskitRuntimeService, Session


import numpy as np
from math import ceil
import time
import sys


# def save_object(obj, filename):
#     with open(filename, 'wb') as outp:
#         pickle.dump(obj, outp, pickle.HIGHEST_PROTOCOL)





# Qiskit runtime backend as a service
# qAccesKey = '<IBM QUANTUM API TOKEN>'
# service = QiskitRuntimeService(channel="ibm_quantum", token=qAccesKey)

# Tuhin's key
qAccesKey = '<IBM QUANTUM API TOKEN>'
service = QiskitRuntimeService(channel="ibm_quantum", token=qAccesKey)

# Set the Sampler and runtime options
# shots_count = int(sys.argv[1])
shots_count = 2000
# options = Options(execution={"shots": 4000})

# backends = [sys.argv[2], sys.argv[3]]
# backends = ['ibmq_qasm_simulator', 'simulator_statevector']
backends = ['ibmq_quito', 'ibmq_belem']

# [New, TK] - Session creation for persistence \
session_dict = {}
for backend in backends:
    session_dict[backend] = Session(service=service, backend=backend)
print(f"CustomDict::SessionDict::{session_dict}")

num_qubits = 6
reps = 1 # we keep this fixed to 1 for the ease of cutting
backend = None

num_vars = RealAmplitudes(num_qubits=num_qubits, reps=reps).num_parameters
init_params = np.random.randn(num_vars)


# Create the provider object - for noise model import
# IBMQ.save_account(qAccesKey)
# IBMQ.load_account() # Load account from disk
# provider = IBMQ.get_provider(hub='ibm-q-education')


# create the backend and save the object into a pickle file
# backend_quito= provider.get_backend("ibmq_quito")
# backend_manila= provider.get_backend("ibmq_manila")
# save_object(backend_quito, "backend_quito.pkl")
# save_object(backend_manila, "backend_manila.pkl")

# create the noise model objects and save into a pickle file
# noise_model_quito = NoiseModel.from_backend(backend_quito)
# noise_model_manila = NoiseModel.from_backend(backend_manila)
# save_object(noise_model_quito , f"noise_model_quito_shots_{shots_count}_with_resilience_unknown.pkl")
# save_object(noise_model_manila , f"noise_model_manila_shots_{shots_count}_with_resilience_unknown.pkl")

# create options with the noise profile
options_noise_r0_backend_1 = Options(
    simulator={
        # "noise_model": noise_model_quito,
        "seed_simulator": 1234,
        "coupling_map": backend_quito.configuration().coupling_map,
        "basis_gates": backend_quito.configuration().basis_gates
    },
    # resilience_level=0,
    execution={"shots": shots_count}
)

options_noise_r0_backend_2 = Options(
    simulator={
        # "noise_model": noise_model_manila,
        "seed_simulator": 1234,
        "coupling_map": backend_manila.configuration().coupling_map,
        "basis_gates": backend_manila.configuration().basis_gates 
    },
    # resilience_level=0,
    execution={"shots": shots_count}
)

options_list = [options_noise_r0_backend_2, options_noise_r0_backend_1]

# Logging variables
cut_timings = []
evaluate_subcircuits_timings = []
post_processing_timings = []
intermediate_energy_values = []

# def get_expval(counts: dict) -> float:
#     '''returns the expval of ZZ...Z type operator; here counts countains probability (not shots)'''
#     expval = 0
    
#     for key, val in counts.items():
#         expval += ((-1)**(key.count('1')%2))*val
    
#     return expval

# def get_expval_zz(counts: dict, i: int, j: int) -> float:
#     '''Returns the ZZ type expectation value given any locations i and j. Typically we will have j=i+1 NN interactions.
#     Here counts contains probability (not shots)'''
    
#     ## note qiskit reads from last bit to first bit. By my convention, i and j follows first to last. So we change it
#     n = len(list(counts.keys())[0]) # finds out the number of qubits
#     i, j = n-i-1, n-j-1
    
#     expval = 0
    
#     for key, val in counts.items():
#         parity = 1 if (int(key[i])+int(key[j]))%2 == 0 else -1
#         expval += parity*val
    
#     return expval

def get_expval_zz(counts: dict, i:int) -> float:
    '''returns the expval of H = Z_i Z_{i+1}'''
    # first make sure that counts contains probabilities, if not make it
    if sum(counts.values()) != 1:
        shots = sum(counts.values())
        for key in counts.keys():
            counts[key] = counts[key]/shots
    
    expval = 0
    
    for key, val in counts.items():
        parity = 1 if (int(key[i])+int(key[i+1]))%2 == 0 else -1
        expval += parity*val
    
    return expval

def get_expval(counts: dict) -> float:
    '''returns the expval of H = \sum_i Z_i Z_{i+1}'''
    num_qubits = len(list(counts.keys())[0])
    expval = 0
    
    for idx in range(num_qubits-1):
        expval += get_expval_zz(counts,idx)
    
    return expval



def cut_and_evaluate_circuit(circuit, max_cuts=2, max_subcircuits=[2]):
    
    cut_start = get_current_timestamp_ms()
    cuts = cut_circuit_wires(
        circuit=circuit,
        method="automatic",
        max_subcircuit_width=ceil(circuit.depth()/2),
        max_cuts=max_cuts,
        num_subcircuits=max_subcircuits,
        verbose = False
        )
    cut_end = get_current_timestamp_ms()
    print(f"cut_and_evaluate::cut_circuit_wires::timings::{(cut_start, cut_end)}")
    cut_time_taken = cut_end - cut_start
    cut_timings.append((cut_start, cut_end, cut_time_taken))

    # subcircuit_instance_probabilities = evaluate_subcircuits(cuts)

    # Uncomment the following lines to instead use Qiskit Runtime Service as configured above.

    eval_subcircuit_start = get_current_timestamp_ms()

    # No Noise model - evaluation (optoins contains just the shot configuration)
    # subcircuit_instance_probabilities = evaluate_subcircuits(cuts,
    #                                                          service=service,
    #                                                          backend_names=backends,
    #                                                          options=options_list,
    #                                                          session_dict=session_dict
    #                                                         )
    
#     program = Program(
#         title="First program",
#         entrypoint="test-vqe-cut.py",
#         working_dir="./source_files/"
#     )

#     job = serverless.run(program, arguments={"circuit1": })
    
    
    subcircuit_instance_probabilities = evaluate_subcircuits(cuts,
                                                             service=service,
                                                             backend_names=backends,
                                                             options=[options_noise_r0_quito, options_noise_r0_belem]
                                                            )    

    eval_subcircuit_end = get_current_timestamp_ms()
    time_taken_eval = eval_subcircuit_end - eval_subcircuit_start
    print(f"cut_and_evaluate::evaluate_subcircuits::timings::{(eval_subcircuit_start, eval_subcircuit_end)}")
    evaluate_subcircuits_timings.append((eval_subcircuit_start, eval_subcircuit_end, time_taken_eval))

    # TODO - time it!
    # NOTE - this is the post processing
    reconstruct_start = get_current_timestamp_ms()
    reconstructed_probabilities = reconstruct_full_distribution(circuit, subcircuit_instance_probabilities, cuts)
    reconstruct_end = get_current_timestamp_ms()
    time_taken_post = reconstruct_end - reconstruct_start
    print(f"cut_and_evaluate::reconstruct::timings::{(reconstruct_start, reconstruct_end)}")
    post_processing_timings.append((reconstruct_start ,reconstruct_end, time_taken_post))


    # NOTE - post processing ends here
    reconstructed_distribution = {i: prob for i, prob in enumerate(reconstructed_probabilities)}
    
    # Represent states as bitstrings (instead of ints)
    reconstructed_dict_bitstring = ProbDistribution(data=reconstructed_distribution).binary_probabilities(num_bits=num_qubits)
    
    
    return reconstructed_dict_bitstring

def cut_objective_function(params):
    print(f"CustomLog[INFO-Params]::{params}")
    ansatz = RealAmplitudes(num_qubits=num_qubits, reps=reps).bind_parameters(params).decompose()
    
    counts = cut_and_evaluate_circuit(ansatz)
    expval = get_expval(counts)
    
    print(f"CustomeLog[INFO-Intermediate-ExpVal]::{expval}")
    intermediate_energy_values.append(expval)
    return expval

def uncut_objective_function(params,shots=2048):
    ansatz = RealAmplitudes(num_qubits=num_qubits, reps=reps).bind_parameters(params).decompose()
    ansatz.measure_all()
    
    counts = backend.run(ansatz, shots=shots).result().get_counts()
    
    for key in counts.keys():
        counts[key] = counts[key]/shots
    
    expval = get_expval(counts)
    
    return expval

optimizer = COBYLA(maxiter=120)

num_vars = RealAmplitudes(num_qubits=num_qubits, reps=reps).num_parameters
init_params = np.random.randn(num_vars)

def get_current_timestamp_ms():
    return time.time()*1000

if __name__ == "__main__":
    optimizer_start = get_current_timestamp_ms()
    mit_cut = optimizer.minimize(fun=cut_objective_function, x0=init_params)
    optimizer_end = get_current_timestamp_ms()
    # mit_uncut = optimizer.minimize(fun=uncut_objective_function, x0=init_params)

    print("Initial parameters: ",init_params)
    print("----------")
    print("energy using circuit cutting: ",mit_cut.fun)
    print("Optimized parameters via circuit cutting: ", mit_cut.x)
    print("----------")
    # print("energy via full circuit evaluation: ",mit_uncut.fun)
    # print("Optimized parameters via full circuit evaluation: ", mit_uncut.x)

    # logs
    print(f"total_optimizer_time::{optimizer_end - optimizer_start}")
    print(f"cut_timings::{cut_timings}")
    print(f"evaluate_timings::{evaluate_subcircuits_timings}")
    print(f"post_process_timings::{post_processing_timings}")
    print(f"intermediate_energy_valyes::{intermediate_energy_values}")
    print(f"Writing output to file..")
    with open("out_logs_vqe_cutqc.log", "w+") as out:
        out.write(f"cut_timings::{cut_timings}\n")
        out.write(f"evaluate_timings::{evaluate_subcircuits_timings}\n")
        out.write(f"post_process_timings::{post_processing_timings}\n")
        out.write(f"intermediate_energy_values::{intermediate_energy_values}\n")
    



In [4]:
import os
os.system("pip install docplex")
os.system("pip install cplex")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 633.5/633.5 kB 10.0 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for docplex: filename=docplex-2.25.236-py3-none-any.whl size=671349 sha256=67aa5cb002021e8c9493aac30857688e7f7370f31d3ae663b5e383699fda3f84
  Stored in directory: /home/jovyan/.cache/pip/wheels/02/32/20/7f3f85d090da2c3b1ec41de83f17f426a05acc5d9f5e8a1c9f
Successfully built docplex
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 2.7 MB/s eta 0:00:00


0

In [6]:
import pickle
from qiskit.circuit.library import RealAmplitudes
from qiskit.algorithms.optimizers import COBYLA
# from qiskit_algorithms import COBYLA
from qiskit.providers.fake_provider import FakeKolkata
# from qiskit_aer import AerSimulator
# from qiskit_aer.noise import NoiseModel
from qiskit.result import ProbDistribution
from qiskit.opflow import I, X, Y, Z
from qiskit_ibm_runtime import Options

from quantum_serverless import Program

# relevant imports for the noise model
from qiskit import IBMQ

# from circuit_knitting_toolbox.circuit_cutting.wire_cutting import cut_circuit_wires, reconstruct_full_distribution, evaluate_subcircuits
from circuit_knitting_toolbox.circuit_cutting.cutqc import cut_circuit_wires, reconstruct_full_distribution, evaluate_subcircuits
# Runtime imports
from qiskit_ibm_runtime import QiskitRuntimeService, Session


import numpy as np
from math import ceil
import time
import sys


In [159]:
from quantum_serverless import Program

# Program for the VQE uncut
# program = Program(
#     title="First program",
#     entrypoint="test-code.py",
#     working_dir="./source_files/"
# )

def get_current_timestamp_ms():
    return time.time()*1000

def get_cuts(circuit, max_cuts=2, max_subcircuits=[2]):
    cut_start = get_current_timestamp_ms()
    cuts = cut_circuit_wires(
        circuit=circuit,
        method="automatic",
        max_subcircuit_width=ceil(circuit.depth()/2),
        max_cuts=max_cuts,
        num_subcircuits=max_subcircuits,
        verbose = False
        )
    cut_end = get_current_timestamp_ms()
    return cuts
    

def cut_objective_function(params):
    print(f"CustomLog[INFO-Params]::{params}")
    ansatz = RealAmplitudes(num_qubits=num_qubits, reps=reps).bind_parameters(params).decompose()
    cuts = get_cuts(ansatz)
    return cuts
    # print(f"CustomeLog[INFO-Intermediate-ExpVal]::{expval}")
    # intermediate_energy_values.append(expval)
    # return expval

num_qubits = 6
reps = 1 # we keep this fixed to 1 for the ease of cutting
num_vars = RealAmplitudes(num_qubits=num_qubits, reps=reps).num_parameters
cuts = cut_objective_function(np.random.randn(num_vars))
print(cuts)


# # Program for VQE cut
program1 = Program(
    title="First program",
    entrypoint="run_subcircuits.py",
    working_dir="./source_files/"
)

program2 = Program(
    title="First program",
    entrypoint="run_subcircuits_whole.py",
    working_dir="./source_files/"
)


# job_cut = serverless.run(program)


CustomLog[INFO-Params]::[-0.90561072  0.01978551  0.9313362  -1.18347487 -0.80585498 -1.58649902
 -0.47436959  0.49903605 -0.21276914 -1.60379368 -0.32631814 -0.43910255]
Exporting as a LP file to let you check the model that will be solved :  inf <class 'float'>
Version identifier: 22.1.1.0 | 2023-02-11 | 22d6266e5
CPXPARAM_Read_DataCheck                          1
CPXPARAM_TimeLimit                               300
Tried aggregator 3 times.
MIP Presolve eliminated 20 rows and 5 columns.
MIP Presolve modified 4 coefficients.
Aggregator did 29 substitutions.
Reduced MIP has 64 rows, 26 columns, and 191 nonzeros.
Reduced MIP has 22 binaries, 4 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.03 sec. (0.41 ticks)
Found incumbent of value 2.000000 after 0.03 sec. (0.45 ticks)
Probing fixed 6 vars, tightened 0 bounds.
Probing changed sense of 10 constraints.
Probing time = 0.01 sec. (0.12 ticks)
Cover probing fixed 0 vars, tightened 2 bounds.
Tried aggregator 3 times.
MIP Presolve el

In [112]:
from quantum_serverless import QuantumServerless, Provider

provider = Provider(
    username="user",
    password="password123",
    host=os.environ.get("GATEWAY_HOST", "http://localhost:8000"),
)

serverless = QuantumServerless(provider)

Attempting to instrument while already instrumented


In [158]:
# job_cut = serverless.run(program, arguments={"subcircuits": cuts['subcircuits']})
import json
job_cut = serverless.run(program, arguments={"subcircuits": cuts})

TypeError: keys must be str, int, float, bool or None, not Qubit

In [180]:
job_cut2 = serverless.run(program2)

In [232]:
job_cut2.status()

'RUNNING'

In [234]:
with open("log.txt", "w") as f:
    f.write(job_cut2.logs())

In [157]:
print(job_cut.logs())

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.6/122.6 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.1/4.1 MB 712.1 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 3.5 MB/s eta 0:00:00
/tmp/ray/session_2023-10-19_19-29-58_124577_1/runtime_resources/working_dir_files/_ray_pkg_98376d424808851c/run_subcircuits.py:12: DeprecationWarning: The package namespace circuit_knitting_toolbox is deprecated and will be removed no sooner than Circuit Knitting Toolbox 0.4.0. Use namespace circuit_knitting instead.
  from circuit_knitting_toolbox.circuit_cutting.cutqc import cut_circuit_wires, reconstruct_full_distribution, evaluate_subcircuits
/tmp/ray/session_2023-10-19_19-29-58_124577_1/runtime_resources/worki

In [ ]:
# # Program for VQE cut
program = Program(
    title="First program",
    entrypoint="test-vqe-cut.py",
    working_dir="./source_files/"
)

job = serverless.run(program)

[Job](https://qiskit-extensions.github.io/quantum-serverless/stubs/quantum_serverless.core.Job.html#quantum_serverless.core.Job) instances have a `status()` method to check status of program execution.

In [8]:
job.status()

'FAILED'

In [15]:
job

<Job | 08f54f7e-6017-48dd-b4b7-bb1a13f4d7af>

`Job` instances also have a `result()` method for retrieving results. The `result()` method will not return until the job is done running the program.

In [17]:
job.result()

{}

To inspect the logs from a program, access them from the ``Job`` instance.

In [10]:
print(dir(job))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_in_terminal_state', '_job_client', 'job_id', 'logs', 'raw_data', 'result', 'status', 'stop']


In [9]:
print(job.logs())

/tmp/ray/session_2023-10-12_21-27-24_597187_1/runtime_resources/working_dir_files/_ray_pkg_0ce438f9b9a67734/test-vqe-cut.py:13: DeprecationWarning: ``qiskit.algorithms`` has been migrated to an independent package: https://github.com/qiskit-community/qiskit-algorithms. The ``qiskit.algorithms`` import path is deprecated as of qiskit-terra 0.25.0 and will be removed no earlier than 3 months after the release date. Please run ``pip install qiskit_algorithms`` and use ``import qiskit_algorithms`` instead.
  from qiskit.algorithms.optimizers import COBYLA
Traceback (most recent call last):
  File "/tmp/ray/session_2023-10-12_21-27-24_597187_1/runtime_resources/working_dir_files/_ray_pkg_0ce438f9b9a67734/test-vqe-cut.py", line 25, in <module>
    from circuit_knitting_toolbox.circuit_cutting.wire_cutting import cut_circuit_wires, reconstruct_full_distribution, evaluate_subcircuits
ModuleNotFoundError: No module named 'circuit_knitting_toolbox'



/tmp/ipykernel_75/2890948178.py:1: DeprecationWarning: The package namespace circuit_knitting_toolbox.circuit_cutting.wire_cutting is deprecated and will be removed in Circuit Knitting Toolbox 0.3.0. Use namespace circuit_knitting_toolbox.circuit_cutting.cutqc instead.
  from circuit_knitting_toolbox.circuit_cutting.wire_cutting import cut_circuit_wires, reconstruct_full_distribution, evaluate_subcircuits


In [11]:
jobs = serverless.get_jobs(limit=10, offset=1)

In [12]:
for job_item in jobs:
    print(job_item, job_item.logs(), job_item.status())
    val = job_item.stop()
    print(val)
        

<Job | 091fab77-7919-4ade-bbf3-29e3683d69a4> No logs yet. STOPPED
Job has been stopped.
<Job | 418c8cfb-7a3a-480b-bea9-3b36854bd6ad> No logs yet. STOPPED
Job has been stopped.
<Job | 7e2c9da7-8a00-4dbe-be38-aa7f26807ce4> No logs yet. STOPPED
Job has been stopped.
<Job | 952ffae9-f276-4f7d-ac30-553bbdf16031> Traceback (most recent call last):
  File "/tmp/ray/session_2023-10-12_19-56-03_067028_1/runtime_resources/working_dir_files/_ray_pkg_90e2b36e1511fda8/test-vqe-cut.py", line 14, in <module>
    from qiskit_algorithms import COBYLA
ModuleNotFoundError: No module named 'qiskit_algorithms'
 STOPPED
Job was already not running.
<Job | 838bf192-d668-4cd1-af54-7f747a5eee8d> /tmp/ray/session_2023-10-12_19-44-06_470856_1/runtime_resources/working_dir_files/_ray_pkg_ff11d79c2a1bf5a8/test-vqe-cut.py:13: DeprecationWarning: ``qiskit.algorithms`` has been migrated to an independent package: https://github.com/qiskit-community/qiskit-algorithms. The ``qiskit.algorithms`` import path is deprecate